In [1]:
import pandas as pd

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [3]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [4]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [5]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)